In [1]:
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [2]:
df = pd.read_csv('geoMatrix1.txt',sep='\t')
df = df.set_index('ID')
df = df.T
gene = ['KIF1B', 'S100A9', 'PGD', 'WDFY3', 'TSPO', 'PADI4', 'SORT1', 'CD63', 'MMP9', 'VNN1', 'ACSL1', 'STAT4', 'ITGAM', 'NOG', 'ALOX5', 'GNS',
'SH3GLB1', 'CYP1B1', 'DRAM1', 'RRAGD', 'NLRC4', 'HIP1', 'FGR', 'STXBP2', 'GCLM', 'SPTLC2', 'BCL2A1', 'LCK', 'CEBPB', 'LAT', 'CTSD', 'CD3E',
'MTF1', 'SLC40A1', 'MLKL', 'HGF', 'PIK3CB', 'ATP6V1C1', 'PTGDR', 'SLC11A1', 'PRKCD', 'GRINA', 'FES', 'CTSW', 'CX3CR1', 'CTSA', 'SCOC', 'G0S2',
'CAMP', 'LY96', 'AIM2', 'CD14','AKR1C3', 'MPO', 'ELANE', 'FHIT', 'THBS1', 'DDIT4']
df = df[gene]
df.to_csv('exp.txt', sep='\t')

In [3]:
df

ID           KIF1B  S100A9       PGD    WDFY3   TSPO      PADI4     SORT1  \
GSM648590  2.58150   2.896  2.240667  1.95650  2.038   2.144000  6.469667   
GSM648591  2.97075   3.287  1.586333  2.03025  2.044   1.833000  1.496333   
GSM648592  5.10200   3.489  2.437667  2.90425  3.501   3.796333  2.506333   
GSM648593  2.83100   3.511  3.477333  2.67925  4.776   4.521333  4.990000   
GSM648594  2.37475   4.295  2.044000  3.24400  3.509   2.339000  1.567667   
...            ...     ...       ...      ...    ...        ...       ...   
GSM648715  2.67175   2.808  3.093333  3.18975  3.616   3.108000  3.884333   
GSM648716  5.40400   3.261  3.656667  4.69775  4.727  10.179333  2.681667   
GSM648717  2.02350   1.298  1.599333  1.96200  2.050   1.841667  1.954667   
GSM648718  1.92775   2.569  2.406667  2.25550  2.805   2.063333  2.005000   
GSM648719  6.85100   4.550  3.201333  6.78800  7.275   4.457667  6.803667   

ID          CD63    MMP9     VNN1  ...    CAMP    LY96    AIM2   CD14  AKR1C3  \
GSM648590  3.661   2.333  19.6100  ...   4.048   6.320  10.370  3.368   0.562   
GSM648591  2.566   5.362  10.3285  ...   1.424   5.178   7.317  2.020   0.712   
GSM648592  3.725  25.040  11.9400  ...   6.942   2.074   1.706  1.251   0.519   
GSM648593  4.463  32.180  24.5700  ...   3.061   3.945   2.474  2.392   0.312   
GSM648594  1.649  10.270  10.1315  ...   7.111   5.736   2.016  4.728   0.248   
...          ...     ...      ...  ...     ...     ...     ...    ...     ...   
GSM648715  3.607  25.830   6.5195  ...   2.031   3.164   1.649  2.893   0.316   
GSM648716  3.496  41.100   3.0850  ...  18.220   1.249   2.068  7.841   0.449   
GSM648717  1.456   8.168   1.3295  ...   0.939   0.590   0.923  2.064   0.292   
GSM648718  2.867   2.876   6.2665  ...   1.124   0.494   1.098  3.061   1.179   
GSM648719  6.819  49.820  23.4950  ...   2.155  10.070  10.660  6.621   4.688   

ID             MPO   ELANE   FHIT      THBS1  DDIT4  
GSM648590   1.2450   1.330  0.215   1.101000  1.447  
GSM648591   1.0640   1.530  0.675   0.829714  1.558  
GSM648592  31.9700  47.270  1.042   4.632571  0.768  
GSM648593   1.6200   2.185  0.385   1.806429  2.573  
GSM648594   4.4505  10.270  1.719   2.404000  0.673  
...            ...     ...    ...        ...    ...  
GSM648715   1.9400   3.341  0.412   1.086429  0.323  
GSM648716  19.3650  70.180  0.994   2.438857  0.570  
GSM648717   1.0245   1.075  1.356   1.370714  0.342  
GSM648718   2.1280   3.364  0.324  25.264143  5.517  
GSM648719   1.7830   1.373  0.321   2.888286  0.463  

[130 rows x 58 columns]

In [24]:
l = pd.read_csv('s.txt',sep='\t')
l

index  label
0    GSM648630      0
1    GSM648631      0
2    GSM648632      0
3    GSM648637      0
4    GSM648638      0
..         ...    ...
125  GSM648715      1
126  GSM648716      1
127  GSM648717      1
128  GSM648718      1
129  GSM648719      1

[130 rows x 2 columns]

In [25]:
df1 = df.reset_index()
df2 = pd.merge(l,df1,how='inner',on='index')
df2

index  label    FGR    MMP9   CD14     HGF  ITGAM  CX3CR1   ELANE  \
0    GSM648630      0  0.771   1.022  0.564  0.9946  0.677  1.5530   0.765   
1    GSM648631      0  1.797   1.235  1.732  1.0292  1.626  2.4455   0.697   
2    GSM648632      0  1.099   2.016  2.073  1.3418  1.114  1.0860   1.224   
3    GSM648637      0  0.677   1.053  0.466  0.9588  0.956  0.5430   1.442   
4    GSM648638      0  0.780   1.180  0.538  0.9486  0.959  0.5970   0.954   
..         ...    ...    ...     ...    ...     ...    ...     ...     ...   
125  GSM648715      1  2.638  25.830  2.893  1.6814  3.263  0.5290   3.341   
126  GSM648716      1  3.999  41.100  7.841  2.3414  8.839  2.8750  70.180   
127  GSM648717      1  1.747   8.168  2.064  1.1552  2.018  0.8755   1.075   
128  GSM648718      1  2.135   2.876  3.061  2.1114  2.023  0.4500   3.364   
129  GSM648719      1  3.223  49.820  6.621  8.4696  6.758  0.6265   1.373   

        LAT   CD3E      LCK  PRKCD      THBS1      MPO  
0    1.0355  0.966  1.06050  0.762   1.183857   0.8310  
1    1.1645  0.880  1.41750  1.535   0.827000   0.8000  
2    0.8210  0.979  0.91450  1.268   1.044857   1.1165  
3    0.6080  0.895  0.45700  0.645   1.017857   1.4600  
4    0.7195  0.908  0.50150  0.656   1.154571   1.4785  
..      ...    ...      ...    ...        ...      ...  
125  0.2480  0.196  0.09345  2.567   1.086429   1.9400  
126  1.2265  0.951  0.90900  7.083   2.438857  19.3650  
127  0.7130  0.962  0.63350  1.843   1.370714   1.0245  
128  0.4915  0.516  0.56550  1.276  25.264143   2.1280  
129  0.3565  0.256  0.23250  3.277   2.888286   1.7830  

[130 rows x 15 columns]

In [26]:
pd.DataFrame(df2['label']).to_csv('label.txt',sep='\t',index=False)

In [28]:
data = pd.read_csv('exp.txt',sep='\t')
data = data.set_index('ID')
def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)
data = mean_norm(data)
target = pd.read_csv('label.txt',sep='\t')

In [29]:
# 划分训练集和测试集
x_train,x_test, y_train, y_test = train_test_split(data,target,test_size=0.2, random_state=0)
x_train = x_train.T
x_train.to_csv('x_train.csv')
x_train = x_train.T

In [16]:
test = pd.read_csv('geoMatrix.txt',sep='\t')
test = test.set_index('geneNames')
test = test.T
feature_importance = pd.read_csv('data_all_merge_RF.csv')
feature_list = feature_importance['ID']
feature_list = feature_list[:26]
test = test.loc[:, feature_list]
test.to_csv('test.txt',sep='\t')